In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
hardware_info_path = "hardware_info.csv"
prom_file_path = "/mnt/sdc/xiaoyu/prom_extr_att.parquet/part-00330-d7afaa0c-ea3a-45f1-9f01-d2d20bffbb9c-c000.snappy.parquet"

In [3]:
hardware_df = pd.read_csv(hardware_info_path)

In [4]:
prom_df = pd.read_parquet(prom_file_path)

In [12]:
prom_df

,id,timestamp,node,node_time_seconds,node_load15,surfsara_power_usage,up,node_netstat_Tcp_OutSegs,node_netstat_Tcp_InErrs,node_context_switches_total,node_load5,node_load1,node_memory_Active_bytes,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InErrors,node_memory_Dirty_bytes,surfsara_ambient_temp,node_netstat_Icmp_InMsgs,node_netstat_Udp_InDatagrams,node_intr_total,node_netstat_Tcp_InSegs,node_memory_Percpu_bytes,node_boot_time_seconds,node_netstat_Udp_OutDatagrams,node_netstat_Icmp_InErrors,node_procs_blocked,node_netstat_Icmp_OutMsgs,node_memory_MemFree_bytes,node_procs_running,node_forks_total,node_hwmon_temp_celsius-min,node_hwmon_temp_celsius-mean,node_hwmon_temp_celsius-max,node_filesystem_avail_bytes-sum,node_filesystem_files-sum,node_network_transmit_bytes_total-sum,node_filesystem_device_error-sum,node_disk_written_bytes_total-sum,node_filesystem_free_bytes-sum,node_thermal_zone_temp-min,node_thermal_zone_temp-mean,node_thermal_zone_temp-max,node_disk_read_bytes_total-sum,nvidia_gpu_memory_used_bytes-sum,nvidia_gpu_temperature_celsius-min,nvidia_gpu_temperature_celsius-mean,nvidia_gpu_temperature_celsius-max,node_arp_entries-sum,nvidia_gpu_fanspeed_percent-min,nvidia_gpu_fanspeed_percent-mean,nvidia_gpu_fanspeed_percent-max,node_filesystem_files_free-sum,nvidia_gpu_power_usage_milliwatts-min,nvidia_gpu_power_usage_milliwatts-sum,nvidia_gpu_power_usage_milliwatts-mean,nvidia_gpu_power_usage_milliwatts-max,node_filesystem_size_bytes-sum,node_disk_writes_completed_total-sum,nvidia_gpu_duty_cycle-min,nvidia_gpu_duty_cycle-mean,nvidia_gpu_duty_cycle-max,node_network_transmit_packets_total-sum,node_udp_queues-sum,node_network_receive_bytes_total-sum,node_network_receive_packets_total-sum,node_network_receive_multicast_total-sum,node_disk_io_now-sum,node_rapl_package_joules_total-sum,node_network_receive_drop_total-sum
118938007,25224420,2022-08-02 14:13:00,r11n5,1.659450e+09,1.00,92.0,1.0,5.297260e+09,0.0,3.665660e+10,1.01,1.0,3.685040e+09,17675800.0,0.0,8192.0,24.0,57.0,55940.0,8.764500e+09,4.663830e+09,14090200.0,1.657200e+09,127809.0,0.0,0.0,64.0,8.850210e+10,17.0,10347400.0,33.0,35.65,45.0,1.880105e+12,230697006.0,2.490132e+13,0.0,1.020846e+12,1.880159e+12,48.0,48.0,48.0,3.950375e+10,0.0,NaN,NaN,NaN,182.0,NaN,NaN,NaN,230052407.0,NaN,0.0,NaN,NaN,1.912248e+12,4082128.0,NaN,NaN,NaN,5.315279e+09,0.0,2.371420e+13,5.282081e+09,2.0,0.0,193230.12,0.0
118938242,25224421,2022-08-02 14:13:30,r11n5,1.659450e+09,1.00,92.0,1.0,5.297260e+09,0.0,3.665660e+10,1.01,1.0,3.686830e+09,17675800.0,0.0,12288.0,24.0,57.0,55940.0,8.764520e+09,4.663830e+09,14090200.0,1.657200e+09,127809.0,0.0,0.0,64.0,8.850220e+10,11.0,10347500.0,33.0,35.65,45.0,1.880105e+12,230697006.0,2.490132e+13,0.0,1.020846e+12,1.880159e+12,49.0,49.0,49.0,3.950375e+10,0.0,NaN,NaN,NaN,182.0,NaN,NaN,NaN,230052407.0,NaN,0.0,NaN,NaN,1.912248e+12,4082133.0,NaN,NaN,NaN,5.315279e+09,0.0,2.371420e+13,5.282081e+09,2.0,0.0,194835.48,0.0
118938492,25224422,2022-08-02 14:14:00,r11n5,1.659450e+09,1.00,92.0,1.0,5.297260e+09,0.0,3.665660e+10,1.01,1.0,3.686210e+09,17675800.0,0.0,8192.0,24.0,57.0,55940.0,8.764530e+09,4.663830e+09,14090200.0,1.657200e+09,127809.0,0.0,0.0,64.0,8.850290e+10,13.0,10347500.0,33.0,35.71,44.0,1.880105e+12,230697006.0,2.490132e+13,0.0,1.020846e+12,1.880159e+12,46.0,46.0,46.0,3.950375e+10,0.0,NaN,NaN,NaN,182.0,NaN,NaN,NaN,230052407.0,NaN,0.0,NaN,NaN,1.912248e+12,4082137.0,NaN,NaN,NaN,5.315279e+09,0.0,2.371420e+13,5.282081e+09,2.0,0.0,196441.06,0.0
118938750,25224423,2022-08-02 14:14:30,r11n5,1.659450e+09,1.00,92.0,1.0,5.297260e+09,0.0,3.665660e+10,1.00,1.0,3.686000e+09,17675800.0,0.0,12288.0,24.0,57.0,55940.0,8.764540e+09,4.663830e+09,14090200.0,1.657200e+09,127809.0,0.0,0.0,64.0,8.850230e+10,13.0,10347500.0,33.0,35.76,45.0,1.880105e+12,230697006.0,2.490132e+13,0.0,1.020846e+12,1.880159e+12,46.0,46.0,46.0,3.950375e+10,0.0,NaN,NaN,NaN,182.0,NaN,NaN,NaN,230052407.0,NaN,0.0,NaN,NaN,1.912248e+12,4082140.0,NaN,NaN,NaN,5.315279e+09,0.0,2.371420e+13,5.282081e+09,2.0,0.0,198

In [14]:
load5_range = prom_df['node_memory_Active_bytes'].describe()
print(load5_range)

count    4.319500e+04
mean     1.098043e+10
std      1.235776e+10
min      1.180890e+09
25%      4.020280e+09
50%      5.395800e+09
75%      1.117455e+10
max      6.383500e+10
Name: node_memory_Active_bytes, dtype: float64


In [21]:
merged_df = pd.merge(prom_df, hardware_df, how='left', left_on='node', right_on='NodeName')

merged_df = merged_df.dropna(subset=['NodeName'])

merged_df['RealMemory'] = merged_df['RealMemory'] * 1e9

for i in range(1, 6):
    merged_df[f'anomaly_rule_{i}'] = 0

rule = 'anomaly_rule_1'
merged_df.loc[merged_df[['node_load15','node_load5','node_load1']].max(axis=1) > 2*merged_df['CPUTot'], rule] = 1
count, ratio = merged_df[rule].sum(), merged_df[rule].mean()
print(f'{rule} anomaly count: {count}, ratio: {ratio}')

rule = 'anomaly_rule_2'
merged_df.loc[merged_df['node_memory_Active_bytes'] > merged_df['RealMemory'], rule] = 1
count, ratio = merged_df[rule].sum(), merged_df[rule].mean()
print(f'{rule} anomaly count: {count}, ratio: {ratio}')

rule = 'anomaly_rule_3'
error_columns = ['node_filesystem_device_error-sum', 'node_netstat_Icmp_InErrors', 
                 'node_netstat_Tcp_InErrs', 'node_netstat_Udp_InErrors']
merged_df.loc[merged_df[error_columns].sum(axis=1) != 0, rule] = 1
count, ratio = merged_df[rule].sum(), merged_df[rule].mean()
print(f'{rule} anomaly count: {count}, ratio: {ratio}')

rule = 'anomaly_rule_4'
merged_df.loc[(merged_df['GPU Model'].isnull() & merged_df['nvidia_gpu_power_usage_milliwatts-max'].notna()) 
              | ((merged_df['GPU Model']=='titanrtx') & (merged_df['nvidia_gpu_power_usage_milliwatts-max'] > 280e3 * merged_df['GPU Count'])) 
              | ((merged_df['GPU Model']=='titanv') & (merged_df['nvidia_gpu_power_usage_milliwatts-max'] > 250e3 * merged_df['GPU Count'])), 
              rule] = 1
count, ratio = merged_df[rule].sum(), merged_df[rule].mean()
print(f'{rule} anomaly count: {count}, ratio: {ratio}')

rule = 'anomaly_rule_5'
merged_df.loc[(merged_df['GPU Model'].isnull() & merged_df['nvidia_gpu_temperature_celsius-max'].notna()) 
              | ((merged_df['GPU Model']=='titanrtx') & (merged_df['nvidia_gpu_temperature_celsius-max'] > 89)) 
              | ((merged_df['GPU Model']=='titanv') & (merged_df['nvidia_gpu_temperature_celsius-max'] > 91)), 
              rule] = 1
count, ratio = merged_df[rule].sum(), merged_df[rule].mean()
print(f'{rule} anomaly count: {count}, ratio: {ratio}')

merged_df['isAnomaly'] = merged_df[[f'anomaly_rule_{i}' for i in range(1, 6)]].max(axis=1)
total_count, total_ratio = merged_df['isAnomaly'].sum(), merged_df['isAnomaly'].mean()
print(f'Total anomaly count: {total_count}, total ratio: {total_ratio}')


anomaly_rule_1 anomaly count: 12242, ratio: 0.04617165960752958
anomaly_rule_2 anomaly count: 0, ratio: 0.0
anomaly_rule_3 anomaly count: 79210, ratio: 0.2987467045835989
anomaly_rule_4 anomaly count: 0, ratio: 0.0
anomaly_rule_5 anomaly count: 11, ratio: 4.148735955585896e-05
Total anomaly count: 86056, total ratio: 0.3245669285399089


In [6]:
merged_df

,id,timestamp,node,node_time_seconds,node_load15,surfsara_power_usage,up,node_netstat_Tcp_OutSegs,node_netstat_Tcp_InErrs,node_context_switches_total,node_load5,node_load1,node_memory_Active_bytes,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InErrors,node_memory_Dirty_bytes,surfsara_ambient_temp,node_netstat_Icmp_InMsgs,node_netstat_Udp_InDatagrams,node_intr_total,node_netstat_Tcp_InSegs,node_memory_Percpu_bytes,node_boot_time_seconds,node_netstat_Udp_OutDatagrams,node_netstat_Icmp_InErrors,node_procs_blocked,node_netstat_Icmp_OutMsgs,node_memory_MemFree_bytes,node_procs_running,node_forks_total,node_hwmon_temp_celsius-min,node_hwmon_temp_celsius-mean,node_hwmon_temp_celsius-max,node_filesystem_avail_bytes-sum,node_filesystem_files-sum,node_network_transmit_bytes_total-sum,node_filesystem_device_error-sum,node_disk_written_bytes_total-sum,node_filesystem_free_bytes-sum,node_thermal_zone_temp-min,node_thermal_zone_temp-mean,node_thermal_zone_temp-max,node_disk_read_bytes_total-sum,nvidia_gpu_memory_used_bytes-sum,nvidia_gpu_temperature_celsius-min,nvidia_gpu_temperature_celsius-mean,nvidia_gpu_temperature_celsius-max,node_arp_entries-sum,nvidia_gpu_fanspeed_percent-min,nvidia_gpu_fanspeed_percent-mean,nvidia_gpu_fanspeed_percent-max,node_filesystem_files_free-sum,nvidia_gpu_power_usage_milliwatts-min,nvidia_gpu_power_usage_milliwatts-sum,nvidia_gpu_power_usage_milliwatts-mean,nvidia_gpu_power_usage_milliwatts-max,node_filesystem_size_bytes-sum,node_disk_writes_completed_total-sum,nvidia_gpu_duty_cycle-min,nvidia_gpu_duty_cycle-mean,nvidia_gpu_duty_cycle-max,node_network_transmit_packets_total-sum,node_udp_queues-sum,node_network_receive_bytes_total-sum,node_network_receive_packets_total-sum,node_network_receive_multicast_total-sum,node_disk_io_now-sum,node_rapl_package_joules_total-sum,node_network_receive_drop_total-sum,NodeName,CPUTot,RealMemory,GPU Model,GPU Count,anomaly_rule_1,anomaly_rule_2,anomaly_rule_3,anomaly_rule_4,anomaly_rule_5,total_anomaly
0,145197417,2022-11-19 10:22:00,r10n5,1.668850e+09,1.01,92.0,1.0,5.746380e+07,0.0,1.303900e+08,1.04,1.10,3.028640e+09,25401.0,0.0,8192.0,24.0,57.0,68474.0,3.206140e+08,5.016030e+07,10813400.0,1.668700e+09,72700.0,0.0,0.0,57.0,9.389490e+10,18.0,360531.0,32.0,34.53,43.0,1.882856e+12,230697094.0,3.400513e+11,0.0,7.329405e+10,1.882910e+12,43.0,43.0,43.0,1.194045e+09,0.000000e+00,NaN,NaN,NaN,81.0,NaN,NaN,NaN,230129577.0,NaN,0.0,NaN,NaN,1.912248e+12,456088.0,NaN,NaN,NaN,5.757665e+07,0.0,3.464853e+11,5.776884e+07,2.0,0.0,153070.58,0.0,r10n5,16.0,9.318400e+13,Unknown,0.0,0,0,0,0,0,0
1,145200297,2022-11-19 10:22:00,r27n10,1.668850e+09,9.13,172.0,1.0,4.566300e+07,0.0,4.774110e+07,9.08,8.27,3.019070e+09,214596.0,0.0,8192.0,25.0,40.0,50225.0,3.132120e+08,3.897360e+07,9502720.0,1.668730e+09,53767.0,0.0,0.0,40.0,9.433920e+10,18.0,231178.0,37.0,42.50,50.0,1.963600e+12,238511616.0,2.021173e+11,0.0,2.837531e+10,1.963653e+12,40.0,45.0,50.0,1.191406e+09,0.000000e+00,NaN,NaN,NaN,139.0,NaN,NaN,NaN,237951084.0,NaN,0.0,NaN,NaN,1.992225e+12,261892.0,NaN,NaN,NaN,4.594714e+07,0.0,1.930470e+11,4.489788e+07,2.0,0.0,261792.00,0.0,r27n10,16.0,9.318400e+13,Unknown,0.0,0,0,0,0,0,0
2,145203177,2022-11-19 10:22:00,r26n29,1.668850e+09,0.01,44.0,1.0,2.726100e+09,0.0,4.422310e+11,0.05,0.02,1.546240e+09,6004410.0,0.0,0.0,24.0,56.0,692793.0,1.152330e+10,2.731650e+09,15335400.0,1.666250e+09,772500.0,0.0,0.0,58.0,9.600560e+10,12.0,123600000.0,21.0,24.39,28.0,1.962777e+12,238511082.0,1.209755e+13,0.0,3.534681e+13,1.962831e+12,23.0,25.5,28.0,1.152401e+10,0.000000e+00,NaN,NaN,NaN,239.0,NaN,NaN,NaN,237950933.0,NaN,0.0,NaN,NaN,1.992225e+12,41347596.0,NaN,NaN,NaN,2.733310e+09,0.0,3.932687e+13,5.593653e+09,11.0,0.0,228320.65,0.0,r26n29,16.0,9.318400e+13,Unknown,0.0,0,0,0,0,0,0
3,145206043,2022-11-19 10:22:00,r10n29,1.668850e+09,4.24,136.0,1.0,1.247370e+06,0.0,2.221670e+08,4.06,4.03,5.423730e+10,24.0,0.0,20480.0,25.0,57.0,37327.0,1.444990e+08,1.219090e+06,9175040.0,1.668790e+09,39519.0,0.0,0.0,57.0,4.396940e+10,20.0,2127130.

In [22]:
columns_to_keep = [ 'timestamp', 'node', 'node_load15', 'node_load5', 'node_load1', 'node_memory_Active_bytes', 
                   'node_filesystem_device_error-sum', 'node_netstat_Icmp_InErrors', 
                   'node_netstat_Tcp_InErrs', 'node_netstat_Udp_InErrors', 
                   'nvidia_gpu_power_usage_milliwatts-max', 
                   'nvidia_gpu_temperature_celsius-max','isAnomaly']

merged_df = merged_df[columns_to_keep]

merged_df.fillna(0, inplace=True)


In [23]:
merged_df

,timestamp,node,node_load15,node_load5,node_load1,node_memory_Active_bytes,node_filesystem_device_error-sum,node_netstat_Icmp_InErrors,node_netstat_Tcp_InErrs,node_netstat_Udp_InErrors,nvidia_gpu_power_usage_milliwatts-max,nvidia_gpu_temperature_celsius-max,isAnomaly
0,2022-11-19 10:22:00,r10n5,1.01,1.04,1.10,3.028640e+09,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2022-11-19 10:22:00,r27n10,9.13,9.08,8.27,3.019070e+09,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2022-11-19 10:22:00,r26n29,0.01,0.05,0.02,1.546240e+09,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2022-11-19 10:22:00,r10n29,4.24,4.06,4.03,5.423730e+10,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2022-11-19 10:22:00,r26n22,16.09,16.11,16.03,5.759390e+09,0.0,15.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
284318,2022-11-04 22:49:30,r12n27,16.02,16.07,16.28,1.606710e+09,0.0,0.0,0.0,0.0,0.0,0.0,0
284319,2022-11-04 22:49:30,r26n14,1.24,1.16,1.05,1.138520e+10,0.0,16.0,0.0,0.0,0.0,0.0,1
284320,2022-11-04 22:49:30,r12n26,1.17,1.11,1.10,1.365930e+10,0.0,7.0,0.0,0.0,0.0,0.0,1
284321,2022-11-04 22:49:30,r28n4,1.69,2.30,4.68,1.748740e+10,0.0,11.0,1.0,0.0,67483.0,45.0,1


In [16]:
merged_df['isAnomaly'].unique()

array([0, 1])

In [20]:
merged_df.to_parquet('threshold_labeled_sample.parquet')

In [9]:
anomaly_rule_4_df = merged_df[merged_df['anomaly_rule_4'] == 1]

related_columns = ['nvidia_gpu_power_usage_milliwatts-max', 'node', 
                   'timestamp']

In [12]:
most_anomalous_node = anomaly_rule_4_df['node'].value_counts().idxmax()

In [17]:
most_anomalous_node_df = anomaly_rule_4_df[anomaly_rule_4_df['node'] == most_anomalous_node].sort_values('timestamp')


In [19]:
most_anomalous_node_df[related_columns]

,nvidia_gpu_power_usage_milliwatts-max,node,timestamp
141714,90549.0,r30n4,2022-11-04 19:21:30
142056,184363.0,r30n4,2022-11-04 19:22:00
142398,84834.0,r30n4,2022-11-04 19:22:30
142740,250248.0,r30n4,2022-11-04 19:23:00
143082,198943.0,r30n4,2022-11-04 19:23:30
...,...,...,...
140071,172175.0,r30n4,2022-11-19 13:48:00
140411,187983.0,r30n4,2022-11-19 13:48:30
140741,172649.0,r30n4,2022-11-19 13:49:00
141065,172078.0,r30n4,2022-11-19 13:49:30


In [22]:
anomaly_rule_4_df['node'].unique()

array(['r30n4', 'r30n2', 'r32n2', 'r31n2', 'r33n3', 'r33n2', 'r31n4',
       'r34n7', 'r30n3', 'r32n5', 'r32n3', 'r28n1', 'r31n5', 'r31n3',
       'r32n7', 'r31n6', 'r32n4', 'r32n1', 'r30n5', 'r32n6', 'r30n1',
       'r31n1'], dtype=object)

In [20]:
merged_df[related_columns]

,nvidia_gpu_power_usage_milliwatts-max,node,timestamp
0,NaN,r10n5,2022-11-19 10:22:00
1,NaN,r27n10,2022-11-19 10:22:00
2,NaN,r26n29,2022-11-19 10:22:00
3,NaN,r10n29,2022-11-19 10:22:00
4,NaN,r26n22,2022-11-19 10:22:00
...,...,...,...
284318,NaN,r12n27,2022-11-04 22:49:30
284319,NaN,r26n14,2022-11-04 22:49:30
284320,NaN,r12n26,2022-11-04 22:49:30
284321,67483.0,r28n4,2022-11-04 22:49:30
